<a href="https://colab.research.google.com/github/shivamkainth/deeplearning/blob/master/Day_3_CNN_Classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning and Applications : Joint Faculty Development Programme
# May 27 -31, 2019 

**Participating Academies : IIT Roorkee ,  MNIT Jaipur, NIT Patna, PDPM IIITDM Jabalpur  **

**Prinicipal Coordinating Academies : IIT Roorkee and PDPM IIITDM Jabalpur**

# Tutorial 3: Introduction to Convolution Neural Networks
** In this tutorial, we provide you a brief introduction to Convolution Neural Networks(CNNs). You will be able to build a CNN model that can be used for classification.**


Convolution Neural Networks are mainly use for large size input data such as Image data. 
* Convolution Neural Networks (CNNs) use parameter sharing.
* Small pattern detectors called filters are used to convolve over the entire image.
* These filters are learned through NN training in the same way as in fully connected networks.
* Just like a hidden layer in a fully connected layer, convolution layers are used in CNNs.
* To handle large size of image data, pooling layers are introduced.
* Normalization layers were used in early CNN architectures, but due to their minimal impact, they are not much used in the present CNNs.


** Today we are going to see a simple 3 convolution layers convolution neural network. We are going to use PlantVillage leaf 
dataset. It contains 38 classes dataset. From which we are just going to use 4 classes of Apple. The objective of the work is to Diseases classification in Apple leaves.**

Dataset Link : :https://github.com/spMohanty/PlantVillage-Dataset/tree/master/raw/color
We split the dataset into training validation and testing sample

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### A. Data Preprocessing

** A1. Import required Libraries **
* From keras library we are going to use image preprocessing task, to normalize the image pixel values in between 0 to 1.
* Model is imported to load variuos Neural NEtwrok models such as Sequential.
* We are going to use Stochastic Gradient Descent(SGD) as a optimizer 
* Keras layers such as Dense, Flatten, Conv2D and MaxPooling is used to implement the CNN model 

In [0]:
import numpy as np
import keras
from keras import models
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import SGD
from keras import layers
from keras.layers import  Dense, Flatten, Conv2D, MaxPooling2D
from keras import Input
                                                                                                            


** A2. Loading the training and testing data and defining the basic parameters **
* We are resizing the input image to 64 * 64

* In the dataset :
    Training Set : 70% 
    Validation Set : 20%
    Test Set : 10%

In [0]:
# Normalize training and validation data in the range of 0 to 1
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Read the training sample and set the batch size 
train_generator = train_datagen.flow_from_directory(
        '/content/gdrive/My Drive/Colab Notebooks/plant_village/train/',
        target_size=(64, 64),
        batch_size=16,
        class_mode='categorical')

# Read Validation data from directory and define target size with batch size
validation_generator = validation_datagen.flow_from_directory(
        '/content/gdrive/My Drive/Colab Notebooks/plant_village/val/',
        target_size=(64, 64),
        batch_size=16,
        class_mode='categorical',
        shuffle=False)

test_generator = test_datagen.flow_from_directory(
        '/content/gdrive/My Drive/Colab Notebooks/plant_village/test/',
        target_size=(64, 64),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)


In [0]:
from PIL import Image
import os, sys


dirs = os.listdir("/content/gdrive/My Drive/Colab Notebooks/plant_village/val/Apple_Frogeye_Spot/")
def resize():
    for item in dirs: 
        if os.path.isfile("/content/gdrive/My Drive/Colab Notebooks/plant_village/val/Apple_Frogeye_Spot/"+item):
            im = Image.open("/content/gdrive/My Drive/Colab Notebooks/plant_village/val/Apple_Frogeye_Spot/"+item)
            f, e = os.path.splitext("/content/gdrive/My Drive/Colab Notebooks/plant_village/val/Apple_Frogeye_Spot/"+item)
            imResize = im.resize((64, 64), Image.ANTIALIAS)
            imResize.save(f + '.jpg', 'JPEG', quality=90)

resize()

### B. Model Building


* We are going to use 2 convolution layers with 3*3 filer and relu as an activation function
* Then max pooling layer with 2*2 filter is used
* After that we are going to use Flatten layer
* Then Dense layer is used with relu function
* In the output layer softmax function is used with 4 neurons as we have four class dataset.
* model.summary() is used to check the overall architecture of the model with number of learnable parameters in each 

#### B1. Model Definition

In [0]:
# Create the model
model = models.Sequential()
# Add new layers
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.summary()

### B2. Compile the model with SGD(Stochastic Gradient Descent) and train it with 10 epochs.

In [0]:
sgd = SGD(lr=0.001,decay=1e-6, momentum=0.9, nesterov=True)
# We are going to use accuracy metrics and cross entropy loss as performance parameters
model.compile(sgd, loss='categorical_crossentropy', metrics=['acc'])
# Train the model 
history = model.fit_generator(train_generator, 
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

### B3. Saving the model


In [0]:
model.save('cnn_classification.h5')

### B4. Loading the Model

In [0]:
model = models.load_model('cnn_classification.h5')

### B5. Saving weignts of model

In [0]:
model.save_weights('cnn_classification.h5')

### B6. Loading the Model weights

In [0]:
model.load_weights('cnn_classification.h5')

### C. Performance Measures

**Now we are going to plot the accuracy and loss **

In [0]:
train_acc = history.history['acc']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [0]:
epochs = range(len(acc)) 
plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()
plt.show()

plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


# Model Testing

In [0]:
# Get the filenames from the generator
fnames = test_generator.filenames
 
# Get the ground truth from generator
ground_truth = test_generator.classes
 
# Get the label to class mapping from the generator
label2index = test_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),test_generator.samples))


# Assignemnt
**You have to load the weights of previous model and with the help of previous weights try to create a CNN model with one more convolution layers. You have to train only after the newly added convolution layers of the neural network. **

Hint : Use model.load_weights('weights.h5', by_name=True)
